notebook to preprocess **[GEO dataset](https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE167880)**, 
"Functional, metabolic and transcriptional maturation of human pancreatic islets derived from stem cells"

datasets: 
- stage 5 in vitro?
- month 1 in vivo?
- month 6 in vivo?

In [1]:
import sys
import os
sys.path.append(os.environ["HOME"]+"/.local/lib/python3.9/site-packages")
import scanpy as sc, anndata as ad
import harmonypy
import leidenalg
# diSNE packages
import numpy as np
import pandas as pd
from scipy.sparse import issparse
import matplotlib.pyplot as plt
import seaborn as sns
import hdf5plugin
from scipy.spatial.distance import pdist, squareform

In [10]:
# import diSNE
import diSNE

In [11]:
%%bash
export PATH=$PATH:/home/$USER/.local/bin

In [16]:
# concatenate all three datasets into single anndata object
dsets = ["GSM5114461_S6_A11", "GSM5114464_S7_D20", "GSM5114474_M3_E7"]
adatas = {}
for ds in dsets:
    print(ds)
    adatas[ds] = sc.read_10x_mtx(DATADIR, prefix=ds+"_", cache=True)
combined = ad.concat(adatas, label="dataset")
combined.obs_names_make_unique()

TypeError: bad operand type for unary -: '_Helper'

In [ ]:
# filter low expression/counts
# filter cells that have less than 200 genes expressed
sc.pp.filter_cells(combined, min_genes=200)
# filter cells with less than 1000 total reads 
sc.pp.filter_cells(combined, min_counts=1000)
# filter genes that are detected in less than 5 cells
sc.pp.filter_genes(combined, min_cells=5)
# genes that have a total count of less than 15
sc.pp.filter_genes(combined, min_counts=15)

In [ ]:
# filter high mitochondria
# compute percent of counts in each cell that are from mitochondrial genes
# annotate the group of mitochondrial genes as "mt"
combined.var["mt"] = combined.var_names.str.startswith("MT-")
sc.pp.calculate_qc_metrics(
    combined, qc_vars=["mt"], percent_top=None, log1p=False, inplace=True
)

In [ ]:
# filter cells with high percentage of counts from mitochondrial genes
# 25% as threshold
# idk what threshold1 is supposed to be
adata_filt = combined[combined.obs.n_genes_by_counts<2500, :]
adata_filt = adata_filt[adata_filt.obs.pct_counts_mt<25, :].copy() 

In [ ]:
# normalize counts
# total-count normalize data matrix to 10,000 reads per cell
sc.pp.normalize_per_cell(adata_filt, counts_per_cell_after=1e4)

In [ ]:
# log transform data
sc.pp.log1p(adata_filt)

In [ ]:
# identify highly variable genes
# options: 
# batch_key="dataset" <- select highly variable genes separately within the 3 datasets
# n_top_genes=500 <- select only the top 500 most variable genes
sc.pp.highly_variable_genes(adata_filt, batch_key="dataset", n_top_genes=500)

In [ ]:
# create new AnnData object for analyses with:
# only highly variable genes
# genes with specific marker genes from paper (manually add back even if not most differentially expressed)
genes = ["GCG", "TTR",  "IAPP",  "GHRL", "PPY", "COL3A1",
    "CPA1", "CLPS", "REG1A", "CTRB1", "CTRB2", "PRSS2", "CPA2", "KRT19", "INS","SST","CELA3A", "VTCN1"]

adata_var = adata_filt[:, (adata_filt.var.index.isin(genes) | adata_filt.var["highly_variable"])]

In [ ]:
# visualize batch effects
# perform PCA on dataset and plot data along the first 2 PCs
# compute first 20 PCs
sc.pp.pca(adata_var, n_comps=20)

In [ ]:
# plot data, color cells based on dataset they came from
sc.pl.pca(adata_var, color="dataset")

In [ ]:
# adjust count data to control for batch effects
# use Harmony from within scanpy
# Import the "external" library
import scanpy.external as sce

# Run harmony using suggested params from the paper
sce.pp.harmony_integrate(adata_var, 'dataset', theta=2, nclust=50,  max_iter_harmony = 10,  max_iter_kmeans=10)

# Reset the original PCs to those computed by Harmony
adata_var.obsm['X_pca'] = adata_var.obsm['X_pca_harmony']

In [ ]:
# identify individual cell types/visualize results
# perform clustering using scanpy
# computes neighborhood graphs. Needed to run clustering.
sc.pp.neighbors(adata_var) 

In [ ]:
# clusters cells based on expression profiles. This is needed to color cells by cluster.
sc.tl.leiden(adata_var) 

In [ ]:
# SCANPY t-SNE VISUALIZATION
# visualize using tSNE
# can change color to color cells by different attributes
# "leiden" = by cluster assignment
sc.tl.tsne(adata_var)
sc.pl.tsne(adata_var, color=['leiden'], legend_loc='on data', legend_fontsize=10, alpha=0.8, size=20)